In [29]:
#| default_exp display_helpers

In [30]:
#| export
from __future__ import annotations

# Helpers

> ...

# Prologue

In [31]:
#| export

import json
import time
import uuid
from collections import deque
from typing import Any
from typing import Mapping
from typing import overload
from typing import Sequence

import fastcore.all as FC
import IPython
import traitlets as T
from fastcore.xml import to_xml
from fasthtml.core import FastHTML
from fasthtml.xtend import Style
from IPython.display import DisplayHandle
from IPython.display import HTML
from olio.common import update_

In [32]:
#| export

from bridget.helpers import skip
from bridget.helpers import kounter

In [33]:
from fastcore.test import *
from IPython.display import display
from loguru import logger
from olio.common import setup_console

----

In [34]:
console, cprint = setup_console(140)

In [35]:
#| exporti

_n = '\n'

----

# Display Helpers

# Rich display

In [36]:
def display_json(json):
    from rich.json import JSON
    from rich.jupyter import display as rich_display
    json_renderable = JSON.from_data(json)
    a = list(console.render(json_renderable))
    rich_display(a, console._render_buffer(a))

In [37]:
display_json({'a': 1, 'b': 2})

{
  "a": 1,
  "b": 2
}

In [38]:
#| export

@overload
def pretty_repr(*o, html:bool=True, text:bool=False, **kwargs) -> str: ...
@overload
def pretty_repr(*o, html:bool=False, text:bool=True, **kwargs) -> str: ...
def pretty_repr(*o, html:bool=True, text:bool=True, **kwargs) -> dict[str, str]|str:
    from rich.pretty import Pretty
    d = Pretty(*o, **kwargs)._repr_mimebundle_(
        include=((),('text/plain',))[text] + ((),('text/html',))[html], 
        exclude=((),('text/plain',))[not text] + ((),('text/html',))[not html]
        )
    return d if len(d) > 1 else tuple(d.values())[0]

In [39]:
display(HTML(pretty_repr({'a': 1, 'b': [1,2,3]}, text=False)))
print(pretty_repr({'a': 1, 'b': [1,2,3]}, html=False))
cprint({'a': 1, 'b': [1,2,3]})

{'a': 1, 'b': [1, 2, 3]}



{'a': 1, 'b': [1, 2, 3]}

In [40]:
#| export

def rich_display(*o, dhdl: DisplayHandle|None=None):
    if not o: return
    vv:tuple[str, ...] = tuple(FC.flatten([_.items() for _ in map(pretty_repr, o)]))  # type: ignore
    dd = {'text/plain':'\n'.join(vv[1::4]), 'text/html':'\n'.join(vv[3::4])}
    if dhdl: dhdl.update(dd, raw=True)
    else: display(dd, raw=True)

In [41]:
rich_display({'a': 1, 'b': 2}, [3, 4, 5])

{'a': 1, 'b': 2}

[3, 4, 5]

In [42]:
dhdl = display(display_id=True)
rich_display({'a': 1, 'b': 2}, [3, 4, 5], dhdl=dhdl)

{'a': 1, 'b': 2}

[3, 4, 5]

# Collapsable JSON

In [43]:
#| export

class RenderJSON(object):
    def __init__(self, json_data, max_height=200, init_level=0):
        if isinstance(json_data, Sequence):
            s = json.dumps(list(json_data))
        elif isinstance(json_data, Mapping):
            s = json.dumps(dict(json_data))
        elif hasattr(json_data, 'to_dict'):
            s = json.dumps(json_data.to_dict())
        elif hasattr(json_data, 'to_json'):
            s = json_data.to_json()
        else:
            s = json_data
        self.json_str = s
        self.uuid = str(uuid.uuid4())
        self.max_height = max_height
        self.init_level = init_level

    def display(self):
        html_content = f"""
<div id="wrapper-{self.uuid}" style="width: 100%; max-height: {self.max_height}px; overflow-y: auto;">
    <div id="{self.uuid}" style="width: 100%;"></div>
    <script>
        function renderMyJson() {{
            renderjson.set_show_to_level({self.init_level});
            document.getElementById('{self.uuid}').appendChild(renderjson({self.json_str}));
        }};
        function loadRenderjson() {{
            if (window.renderjson) return Promise.resolve();
            return new Promise((resolve, reject) => {{
                const script = document.createElement('script');
                script.src = 'https://cdn.jsdelivr.net/npm/renderjson@latest/renderjson.js';
                script.onload = resolve;
                script.onerror = reject;
                document.head.appendChild(script);
            }});
        }};
        loadRenderjson().then(renderMyJson).catch(err => console.error('Failed to load renderjson:', err));
    </script>
</div>
"""
        display(HTML(html_content), metadata={'bridge': {'skip':True}})

    def _ipython_display_(self):
        self.display()

In [44]:
json_data = {
    "name": "Petronila",
    "age": 28,
    "interests": ["reading", "cycling", "technology"],
    "education": {
        "bachelor": "Computer Science",
        "master": "Data Science",
        "phd": "Not enrolled"
    }
}

RenderJSON(json_data, init_level=1).display()

# CLog

In [45]:
#| export

def CLog(*o):
    return f"<script>console.log({','.join(map(repr, o))})</script>"

In [46]:
display(HTML(CLog('aaaa', 'bbbb')))

# displaydh

In [47]:
#| export

@FC.delegates(display, keep=True)  # type: ignore
def displaydh(*objs, **kwargs) -> DisplayHandle:
    return display(*objs, **update_(kwargs, display_id=True))  # type: ignore
displaydh.__doc__ = IPython.display.display.__doc__  # type: ignore

In [48]:
#| export

@overload
def display(
    *objs, include=None, exclude=None, metadata=None, transient=None,
    display_id:bool=True,
    raw=False, clear=False, **kwargs) -> DisplayHandle: ...
@overload
def display(
    *objs, include=None, exclude=None, metadata=None, transient=None,
    display_id=str|None,
    raw=False, clear=False, **kwargs) -> None: ...
def display(
    *objs, include=None, exclude=None, metadata=None, transient=None,
    display_id=None,
    raw=False, clear=False, **kwargs) -> DisplayHandle | None: ...

display = IPython.display.display  # type: ignore

In [49]:
dh = display('a', display_id=True)

'a'

# BasicLogger

In [50]:
#| export

class NBLogger:
    def log(self, *msg:Any, clear:bool=False):...
    def clear_log(self):...
    def close(self, msg:str|None=None):...
    def display(self, content:str=''):...


class NoopLogger(NBLogger):
    def __getattr__(self, name:str): return FC.noop

class BasicLogger(NBLogger):
    _fmt = str
    _stl = 'border: 1px solid black; overflow: auto;'
    def __init__(self, msg:str|None=None, **kwargs):
        self.setup(msg, **kwargs)
    def setup(self, msg:str|None=None, height:int=200):
        self.html = False
        self._msgs = deque(maxlen=30)
        self.height = height
        self.dh = displaydh(display_id=True, metadata=skip())
        if msg: self.log(msg)
        return self
    
    def display(self, content:str=''):
        if self.dh: 
            try: self.dh.update(HTML(''), metadata=skip())
            except Exception: pass
        self.dh = displaydh(display_id=True, metadata=skip())
        self.log(content)

    def log(self, *msg:Any, clear:bool=False):
        if clear: self._msgs.clear()
        self._msgs.appendleft(' '.join(map(self._fmt, msg)))
        ss = '\n'.join((f"<div>{msg}</div>" for msg in self._msgs) if not self.html else self._msgs)
        if self.dh: 
            try: self.dh.update(HTML(f"<div style='max-height: {self.height}px; {self._stl}'>{ss}</div>"), metadata=skip())
            except Exception: pass
    def clear_log(self):
        self._msgs.clear()
        if self.dh: 
            try: self.dh.update(HTML(''), metadata=skip())
            except Exception: pass
    def close(self, msg:str|None=None):
        if self.dh and msg: 
            try: self.dh.update(HTML(msg), metadata=skip())
            except Exception: pass
        self.dh = None

In [51]:
bl = BasicLogger('BasicLogger initialized', height=100)

In [52]:
for i,x in enumerate(range(10)): bl.log(f'test{i}')

In [53]:
bl.display()

In [54]:
bl.close('BasicLogger closed')

In [55]:
#| exporti

level_colors = {
    "DEBUG": "#666666",    # gray
    "INFO": "#000000",     # black
    "SUCCESS": "#007700",  # green
    "WARNING": "#ff8800",  # orange
    "ERROR": "#ff0000",    # red
    "CRITICAL": "#880000", # dark red
}

In [56]:
class LoguruBasicLogger(BasicLogger):
    def __init__(self): 
        super().__init__()
        self._fmt = FC.noop
    
    def write(self, message: str) -> None:
        if rec := getattr(message, 'record', None):
            level = rec['level'].name
        else: 
            for level in level_colors: 
                if level in message: break
        message = f"<span style='color: {level_colors[level]}'>{message}</span>"
        self.log(message)

In [57]:
logger.remove()  # Remove default handler
handler_id = logger.add(
    (lbl := LoguruBasicLogger()).write, 
    format="{level} | {message}",  # Simple format, we'll add HTML in the sink
    colorize=False  # Disable ANSI colors
)

In [58]:
logger.debug("This is a debug message")
logger.info("This is an info message")
logger.warning("This is a warning")
logger.error("This is an error")

In [59]:
class LoguruBasicLogger(BasicLogger):
    def __init__(self):
        super().__init__()
        self._fmt = FC.noop
        self.html = True
    
    def write(self, message:str) -> None:
        rec = getattr(message, 'record', json.loads(message))
        formatted_msg = (
            f"<div style='display: flex; gap: 8px'>"
            f"<span style='color: #888'>{rec['time'].strftime('%H:%M:%S')}</span>"
            f"<span style='color: {level_colors[rec['level'].name]}'>{rec['level'].name:8}</span>"
            f"<span>{rec['message']}</span>"
            f"</div>"
        )
        
        self.log(formatted_msg)

def configure_logger(basic_logger: BasicLogger) -> int:
    """Configure loguru to use a specific BasicLogger instance."""
    logger.remove()
    return logger.add(
        basic_logger.write,  # type: ignore
        serialize=True  # This makes loguru pass a json to write()
    )

In [60]:
lbl = LoguruBasicLogger()
handler_id = configure_logger(lbl)

In [61]:
logger.debug("This is a debug message")
logger.info("This is an info message")
logger.warning("This is a warning")
logger.error("This is an error")

In [62]:
#| exporti

@FC.patch
def setup_loguru_sink(self: BasicLogger, logger):
    def sink(message:str) -> None:
        rec = getattr(message, 'record', None) or json.loads(message)
        formatted_msg = (
            f"<div style='display: flex; gap: 8px'>"
            f"<span style='width: 8em; color: #888'>{rec['time'].strftime('%H:%M:%S.%f')}</span>"
            f"<span style='width: 3.3em; color: {level_colors[rec['level'].name]}'>{rec['level'].name.lower():8}</span>"
            # f"<span>{rec['level'].icon}</span>"
            f"<span>{rec['message']}</span>"
            f"</div>"
        )
        
        self.log(formatted_msg)
    
    self.html = True
    logger.remove()
    logger.add(
        sink, 
        # format="<span style='color: {level.color}'>{message}</span>",
        # colorize=True
        # serialize=True
    )
    return sink

In [63]:
my_logger = BasicLogger().setup_loguru_sink(logger)  # type: ignore

In [64]:
logger.debug("This is a debug message")
logger.info("This is an info message")
logger.warning("This is a warning")
logger.error("This is an error")

# Canvas

In [65]:
#| export

LogCanvas_css = '''
  @scope (.brd-logger) {
    :scope { 
      border: 1px solid steelblue; 
      overflow: auto; 
      font-family: monospace; 
      font-size: 13px; 
    }
    .ts { color:lightgray; }
  }
'''
LogCanvas_stl = Style(LogCanvas_css, id='brd-logger-stl')

In [66]:
display(HTML(to_xml(LogCanvas_stl)))

In [67]:
#| export

class Canvas(T.HasTraits):
    height = T.Int(200).tag(sync=True)
    elid = T.Unicode('').tag(sync=True)
    def display(self): ...

class LogCanvas(Canvas):
    _stl = 'border: 1px solid black; overflow: auto; font-family: monospace; font-size: 13px;'
    
    def display(self, content:str=''):
        elid = f"brd-logger-{kounter('canvas')}"
        display(HTML(
            f"<div id='{elid}' class='brd-logger' "
            f"style='width: 100%; max-height: {self.height}px;'"# {self._stl}'"
            f">{content}</div>"))
        time.sleep(0.25)
        self.elid = elid

In [68]:
canvas = LogCanvas(height=100)
canvas.display(f"{'<br>'.join('abc')}<br>")

In [69]:
canvas.display(f"<span class='ts'>ts</span> <span class='msg'>msg</span>")

In [70]:
canvas.display(f"{'<br>'.join('def')}<br>")

# nb_app
> Basic naked FastHTML app

In [63]:
#| export

@FC.delegates(FastHTML)  # type: ignore
def nb_app(**kwargs):
    from starlette.middleware.cors import CORSMiddleware
    kwargs.update(default_hdrs=False, sess_cls=None)
    app = FastHTML(**kwargs)
    app.user_middleware = list(filter(lambda x: x.cls is not CORSMiddleware, app.user_middleware))
    return app


# Colophon
----

In [71]:
import fastcore.all as FC
import nbdev
from nbdev.clean import nbdev_clean


In [72]:
if FC.IN_NOTEBOOK:
    nb_path = '02_display_helpers.ipynb'
    # nbdev_clean(nb_path)
    nbdev.nbdev_export(nb_path)
